In [1]:
!pip install galois
!pip install scipy

In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats

In [2]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [3]:
#definisco una matrice k x (n-k)
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [4]:
#FACCIO METODO DI MONTECARLO
#numero iterazioni Monte Carlo
num_iterazioni = 1000  

def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def duplicati(etichetta):
    #se torna true --> lunghezze diverse --> ci sono duplicati
    return len(etichetta) != len(set(etichetta))

#fa il procedimento di ordinamento tra le righe (che pero sono già ordinate al loro interno)
def ordina_matrice(matrice):
    print("Entro in ordina matrice")
    start = time.perf_counter()
    ord = []
    for riga in matrice:
        ord.append(np.sort(riga))
    ord = np.array(ord)
    #print(ord)
    
    etichetta = np.array([hash(tuple(riga.tolist())) for riga in ord])

    #voglio vedere durata esecuzione con meno colonne
    if not duplicati(etichetta):
        indici = np.argsort(etichetta)
        end = time.perf_counter()
        #finisce confronto con colonne ridotte
        tempo = (end-start)*1000
        return True, indici, tempo
    #end = time.perf_counter()
    #finisce confronto con colonne ridotte
    #tempo = (end-start)*1000
    return False, [], 0.0
    
# Funzione per ordinare le righe con contatori di swap. In num_confronti metto confronti in # sufficienti metto confronti1. 
def ordina_per_riga_con_contatori(matrice, num_swap_righe):
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    print("Funzione ordina_per_riga")
    posizioni_originali = np.arange(matrice.shape[0])
    
    ris_confronto, indici_ordinamento, tempo = ordina_matrice(matrice)
    #print(etichetta[indici_ordinamento])
    #se torno true quindi due matrici sono uguali
    if ris_confronto:
        # Conta gli swap solo quando la posizione delle righe cambia
        for i, indice in enumerate(indici_ordinamento):
            if indice != posizioni_originali[i]:
                num_swap_righe += 1
        return matrice[indici_ordinamento], num_swap_righe, tempo
    else:
        return [], num_swap_righe, tempo


#POTREI DOVER IMPLEMENTARE UNA COSA SIMILE A SOPRA PER IL COUNT DEI CONFRONTI (LEXSORT NON CE L'HA)
#ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    #riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    return matrice_ordinata, num_swap_colonne

def test_algoritmo(matrice1, num_swap_righe, num_swap_colonne):
    print("Funzione test uguaglianza")
    start_time = time.perf_counter()
    #intervallo corretto per le righe
    righe = range(k)      
    
    print("Faccio ordina per riga con contatori")
    ordinata1, num_swap_righe, tempo = ordina_per_riga_con_contatori(matrice1, num_swap_righe)
    print("Ordino le colonne")
    #se ordinata è praticamente vuoto vuol dire che ordina per riga per contatori non ha passato qualche test
    if len(ordinata1) == 0:
        end_time = time.perf_counter()
        durata = (end_time - start_time)*1000
        return False, num_swap_righe, num_swap_colonne, durata, tempo
    

    cf1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    return True, num_swap_righe, num_swap_colonne, durata, tempo

#Funzione per eseguire la simulazione di Monte Carlo
def simulazione_montecarlo(num_iterazioni):
    #contatore simulazione accettabile
    simulazione_accettabile = 0
    #contatore swap righe
    num_swap_righe = 0  
    #contatore swap colonne
    num_swap_colonne = 0  
    #contatore confronti righe, dove c'è sufficienti è perchè si riferisce a test con meno colonne
    confronti_per_test = []
    sufficienti_per_test = []
    num_confronti_accumulatore = 0
    conta_sufficienti_accumulatore = 0
    
    #inizializzo la durata
    tempo_esecuzione = 0.0
    durata = 0
    tempo_hash = 0.0
    num_confronti = 0
    conta_sufficienti = 0
    accumula_durata_tutte_colonne = 0.0
    accumula_durata_meno_colonne = 0.0

    
    for z in range(num_iterazioni):
        print(f"ITERAZIONE {z}")
        #field.order mi fa generare i numeri da 0 a 7 (field.order = 8)
        A = field(np.random.randint(0, field.order, size=(k, n_k)))
        while(has_duplicate_rows(A)):
                A = field(np.random.randint(0, field.order, size=(k, n_k)))    
        esito, num_swap_righe, num_swap_colonne, durata, tempo = test_algoritmo(A, num_swap_righe, num_swap_colonne)
        """confronti_per_test = np.append(confronti_per_test,num_confronti)
        sufficienti_per_test = np.append(sufficienti_per_test,conta_sufficienti)
        accumula_durata_tutte_colonne += tempo
        accumula_durata_meno_colonne += tempo1
        num_confronti_accumulatore += num_confronti
        conta_sufficienti_accumulatore += conta_sufficienti"""
        tempo_hash += tempo
        #calcolo il tempo di ogni esecuzione, pure quelle negative
        tempo_esecuzione += durata
        #conto solo quelle che hanno successo anche con meno colonne
        if esito:
            simulazione_accettabile += 1
        
    #H0 è che la differenza tra numero dei confronti con tutte le colonne e quella con le colonne limitate NON è significativa
    """t_stat, p_value = stats.ttest_rel(confronti_per_test, sufficienti_per_test)
    print(f"T-statistica: {t_stat}, P-value: {p_value}")
    alpha = 0.05
    if p_value < alpha:
        print("La differenza tra confronti normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")"""
    #Faccio num_confronti_accumulatore/num_iterazioni, conta_sufficienti_accumulatore/num_iterazioni perchè in ogni caso
    #sia che la simulazione sia accettabile (ordinamento abbreviato = quello intero) che non, comunque conto il numero di
    #swap. Effettivamente potrei provare a fare un contatore del tempo solo per i casi positivi
    return tempo_esecuzione/num_iterazioni, tempo_hash/num_iterazioni, num_swap_righe/num_iterazioni, num_swap_colonne/num_iterazioni, (simulazione_accettabile/num_iterazioni)*100    



In [5]:
# Output statistiche
# Esegui la simulazione
tempo_es_medio, tempo_hash_medio, num_medio_sw_ri, num_medio_sw_co, accettati_perc = simulazione_montecarlo(num_iterazioni)

ITERAZIONE 0
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 1
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 2
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 3
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 4
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 5
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Fu

Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 9
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 10
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 11
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 12
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 13
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 14
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_pe

In [6]:
print()
print(f"Tempo esecuzione medio: {tempo_es_medio} ms")
print(f"Tempo hash medio: {tempo_hash_medio} ms")
print(f"Numero medio di swap righe: {num_medio_sw_ri}")
print(f"Numero medio di swap colonne: {num_medio_sw_co}")
print(f"Percentuale di accettati: {accettati_perc}%")


Tempo esecuzione medio: 13.02970015900155 ms
Tempo hash medio: 9.083381245003693 ms
Numero medio di swap righe: 125.006
Numero medio di swap colonne: 125.009
Percentuale di accettati: 100.0%


In [7]:
nome_file = "stats_hash.txt"

with open(nome_file, 'a') as file:
    file.write(f"Numero iterazioni: {num_iterazioni}\n")
    file.write(f"Tempo esecuzione medio: {tempo_es_medio} ms\n")
    file.write(f"Tempo hash medio: {tempo_hash_medio} ms\n")
    file.write(f"Numero medio di swap righe: {num_medio_sw_ri}\n")
    file.write(f"Numero medio di swap colonne: {num_medio_sw_co}\n")
    file.write(f"Percentuale di accettati: {accettati_perc}%\n\n")    
